In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

In [3]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


# Q1

In [4]:
len(df.columns)

19

# Q2

In [5]:
df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds()/60

In [6]:
round(df['duration'].std(), 2)

42.59

# Q3

In [7]:
df_xoutliers = df[(df['duration'] >= 1) & (df['duration'] <= 60)]

In [8]:
round(len(df_xoutliers)/len(df) * 100, 2)

98.12

# Q4

In [9]:
df_xoutliers['PULocationID'] = df_xoutliers['PULocationID'].astype(str)
df_xoutliers['DOLocationID'] = df_xoutliers['DOLocationID'].astype(str)

/tmp/ipykernel_19581/1499863208.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xoutliers['PULocationID'] = df_xoutliers['PULocationID'].astype(str)
/tmp/ipykernel_19581/1499863208.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xoutliers['DOLocationID'] = df_xoutliers['DOLocationID'].astype(str)


In [10]:
from sklearn.feature_extraction import DictVectorizer

In [11]:
features = df_xoutliers[['PULocationID', 'DOLocationID']].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(features)

In [12]:
X_train.shape[1]

515

# Q5

In [13]:
y_train = df_xoutliers['duration'].values

In [14]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [15]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [16]:
y_pred_train = lr.predict(X_train)
rmse_train = mean_squared_error(y_train, y_pred_train, squared=False)

In [17]:
round(rmse_train, 2)

7.65

# Q6

In [18]:
df1 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

df1['duration'] = (df1['tpep_dropoff_datetime'] - df1['tpep_pickup_datetime']).dt.total_seconds() / 60
df1 = df1[(df1['duration'] >= 1) & (df1['duration'] <= 60)]

df1['PULocationID'] = df1['PULocationID'].astype(str)
df1['DOLocationID'] = df1['DOLocationID'].astype(str)

features1 = df1[['PULocationID', 'DOLocationID']].to_dict(orient='records')
X_train1 = dv.transform(features1)
y_train1 = df1['duration'].values

y_pred1 = lr.predict(X_train1)
rmse_val = mean_squared_error(y_train1, y_pred1, squared=False)

round(rmse_val, 2)

7.81